In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
torch.__version__

'1.7.1+cu101'

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
device

'cuda'

In [5]:
data_path = '../data'

In [6]:
train_image_path = os.path.join(data_path, 'train')
test_image_path = os.path.join(data_path, 'test')

In [7]:
train_table = pd.read_csv("../data/train.csv")

In [8]:
train_file_paths = glob.glob(os.path.join(data_path, 'train', '*'))

In [9]:
train_table.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [ ]:
fig, ax = plt.subplots(3,10, figsize=(15, 6))
r = 0
c = 0
for idx in train_table.index[:30]:
    arr_img = np.array(Image.open(os.path.join(data_path, "train", train_table.loc[idx, "id"])))
    ax[r, c].imshow(arr_img)
    ax[r, c].set_title("has_cactus: " + str(train_table.loc[idx, "has_cactus"]))
    ax[r, c].axis("off")
    c += 1
    if c % 10 == 0:
        c = 0
        r += 1

In [ ]:
arr_img.shape

### preprocessing

In [ ]:
def preprocessing(img):
    img = Image.open(img)
#     img_grey = img.convert('L')
    
    return img

In [ ]:
im = preprocessing(os.path.join(train_image_path, train_table.loc[1, "id"]))

In [ ]:
np.array(im).shape

### modeling

In [10]:
train_table = train_table.sample(frac=1)

In [11]:
X, y = train_table["id"].values, train_table["has_cactus"].values

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([ 3504, 10496]))

In [14]:
np.unique(y_test, return_counts=True)

(array([0, 1]), array([ 860, 2640]))

In [15]:
class CactusDataset(Dataset):
    def __init__(self, X_train, X_test, y_train, y_test):
        self.dataset = {
            "train": (X_train, y_train, len(y_train)),
            "test": (X_test, y_test, len(y_test))
        }
        self.set_split(split="train")
        
    def set_split(self, split="train"):
        self.data_x, self.data_y, self.length = self.dataset[split]
    
    def preprocessing(self, filename):
        path = os.path.join(train_image_path, filename)
        img = Image.open(path)
            
        im_arr = np.array(img) / 255
        im_arr = im_arr.transpose(2,0,1)
        
        return im_arr
    
    def __getitem__(self, idx):
        x = torch.Tensor(self.preprocessing(self.data_x[idx]))
        y = torch.Tensor([self.data_y[idx]])
        
        return x, y

    def __len__(self):
        return self.length

In [16]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        
        self.network = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.ReLU(),
            nn.Conv2d(16, 128, 3),
            nn.ReLU(),
            nn.Conv2d(128, 256, 3),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3),
            nn.ReLU(),
            nn.Conv2d(256, 1, 3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(484, 1),
        )
        
    def forward(self, input_):
        out = self.network(input_)
        
        return out

In [17]:
dataset = CactusDataset(X_train, X_test, y_train, y_test)

In [18]:
model = Classifier().to(device)

In [19]:
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
criterion = nn.BCELoss()

In [20]:
## sum weight + bias
num_param = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {num_param:,}")

Total number of parameters: 907,046


In [21]:
def compute_accuracy(y, out):
    out_indicies = (out > 0.5).long()
    y = y.long()
    n_correct = torch.eq(y, out_indicies).sum().item()
    accuracy = n_correct / y.shape[0]
    
    return accuracy * 100

In [22]:
data_sample = DataLoader(dataset, batch_size = 2)
for data in data_sample:
    break

In [23]:
for epoch in range(1, 101):
    
    running_loss = 0
    running_loss_v = 0
    running_acc = 0
    running_acc_v = 0
    
    dataset.set_split("train")
    data_gen = DataLoader(dataset, batch_size=1024, shuffle=True)
    model.train()
    for batch_index, (x, y) in enumerate(data_gen, 1):
        
        optimizer.zero_grad()
        
        x = x.to(device)
        y = y.to(device)
        
        out_logit = model(x)
        out = torch.sigmoid(out_logit)
        
        loss = criterion(out, y)
        loss_train = loss.item()
        running_loss += (loss_train - running_loss) / batch_index
        
        accuracy = compute_accuracy(y, out)
        running_acc += (accuracy - running_acc) / batch_index
        
        loss.backward()
        optimizer.step()
        
    dataset.set_split("test")
    data_gen = DataLoader(dataset, batch_size=1024)
    model.eval()
    for batch_index, (x, y) in enumerate(data_gen, 1):
        
        x = x.to(device)
        y = y.to(device)
        
        with torch.no_grad():
            out = model(x)
            out = torch.sigmoid(out)
        
        loss = criterion(out, y)
        loss_val = loss.item()
        running_loss_v += (loss_val - running_loss_v) / batch_index
        
        accuracy = compute_accuracy(y, out)
        running_acc_v += (accuracy - running_acc_v) / batch_index
    
    print(f'epoch: {epoch}')
    print(f'\ttrain loss: {running_loss:.2f} | accuracy: {running_acc:.2f}')
    print(f'\tval loss: {running_loss_v:.2f} | accuracy: {running_acc_v:.2f}')

epoch: 1
	train loss: 0.63 | accuracy: 74.94
	val loss: 0.56 | accuracy: 76.15
epoch: 2
	train loss: 0.58 | accuracy: 74.94
	val loss: 0.56 | accuracy: 76.15
epoch: 3
	train loss: 0.57 | accuracy: 75.00
	val loss: 0.55 | accuracy: 76.15
epoch: 4
	train loss: 0.55 | accuracy: 74.91
	val loss: 0.53 | accuracy: 76.15
epoch: 5
	train loss: 0.53 | accuracy: 74.97
	val loss: 0.49 | accuracy: 76.15
epoch: 6
	train loss: 0.47 | accuracy: 74.98
	val loss: 0.41 | accuracy: 76.15
epoch: 7
	train loss: 0.39 | accuracy: 75.11
	val loss: 0.35 | accuracy: 76.41
epoch: 8
	train loss: 0.35 | accuracy: 75.69
	val loss: 0.32 | accuracy: 76.86
epoch: 9
	train loss: 0.33 | accuracy: 75.85
	val loss: 0.31 | accuracy: 76.86
epoch: 10
	train loss: 0.32 | accuracy: 75.87
	val loss: 0.31 | accuracy: 76.93
epoch: 11
	train loss: 0.32 | accuracy: 76.08
	val loss: 0.31 | accuracy: 77.02
epoch: 12
	train loss: 0.31 | accuracy: 76.11
	val loss: 0.30 | accuracy: 77.18
epoch: 13
	train loss: 0.31 | accuracy: 76.09
	va